In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import models
import matplotlib.pyplot as plt

import numpy as np
import wandb
import pandas as pd
import cv2
from tqdm import tqdm
import argparse
from PIL import Image

from sklearn.metrics import f1_score, precision_score, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class Model_EfficientNet(nn.Module):
    def __init__(self, model_type):
        super(Model_EfficientNet, self).__init__()

        if model_type == "EfficientNet_v2_s":
            self.model = models.efficientnet.efficientnet_v2_s(pretrained=True)
        elif model_type == "EfficientNet_v2_m":
            self.model = models.efficientnet.efficientnet_v2_m(pretrained=True)
        elif model_type == "EfficientNet_v2_l":
            self.model = models.efficientnet.efficientnet_v2_l(pretrained=True)
        elif model_type == "EfficientNet_B0":
            self.model = models.efficientnet.efficientnet_b0(pretrained=True)
        elif model_type == "EfficientNet_B1":
            self.model = models.efficientnet.efficientnet_b1(pretrained=True)
        elif model_type == "EfficientNet_B2":
            self.model = models.efficientnet.efficientnet_b2(pretrained=True)
        elif model_type == "EfficientNet_B3":
            self.model = models.efficientnet.efficientnet_b3(pretrained=True)
        elif model_type == "EfficientNet_B4":
            self.model = models.efficientnet.efficientnet_b4(pretrained=True)
        elif model_type == "EfficientNet_B5":
            self.model = models.efficientnet.efficientnet_b5(pretrained=True)
        elif model_type == "EfficientNet_B6":
            self.model = models.efficientnet.efficientnet_b6(pretrained=True)
        elif model_type == "EfficientNet_B7":
            self.model = models.efficientnet.efficientnet_b7(pretrained=True)
        self.model.classifier[1] = nn.Linear(2560, 10, bias=True)
    def forward(self, x):
        x = self.model_backbone(x)
        return x

/home/nabang1010/anaconda3/envs/nabang_datn/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/nabang1010/anaconda3/envs/nabang_datn/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B7_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B7_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b7_lukemelas-dcc49843.pth" to /home/nabang1010/.cache/torch/hub/checkpoints/efficientnet_b7_lukemelas-dcc49843.pth


  0%|          | 0.00/255M [00:00<?, ?B/s]

In [4]:
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
            (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(64, 16, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormAct

In [ ]:
# augmentation
transforms = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((32, 32)),
        transforms.RandomRotation(10),
        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,)),
    ]
)